Running with Colab environment

In [ ]:
import openai
from openai import OpenAIError
import time
import base64

openai.api_key = ''  # Your OpenAI API key here


def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def get_chatgpt_response_image(text_data, encodedImage, prompt):
    retry_attempts = 5
    for attempt in range(retry_attempts):
        try:
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{encodedImage}",
                                "detail": "auto"
                            }
                        },
                        {"type": "text", "text": text_data}
                    ]
                }
            ]

            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            return response.choices[0].message.content

        except OpenAIError as e:
            if attempt < retry_attempts - 1:
                wait_time = 2 ** attempt
                print(f"Over speed processing {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

input_text = "Ảnh chụp di tích lịch sử Khám Lớn Cần Thơ."
tag = "giá trị văn hóa – lịch sử"

prompt = (
    "Dựa vào bức ảnh được cung cấp, hãy tạo ra các cặp câu hỏi và câu trả lời phù hợp nhất, liên quan trực tiếp đến di sản văn hóa cách mạng được quan sát từ hình ảnh. Với yêu cầu sau: "
    "1. Với tag là " + tag + ", câu hỏi và câu trả lời cần tập trung khai thác, làm rõ các yếu tố quan sát được."
    "2. Không thêm dữ liệu bên ngoài ảnh và không suy diễn về ý nghĩa hay lịch sử."
    "3. Trả lời bằng định dạng JSON: 'qas': [{'question': ..., 'answer': ...}].
)

image = "images/can_tho/kham_lon_can_tho/19.jpg"

encodedImage = encode_image_to_base64(image)

result = get_chatgpt_response_image(input_text, encodedImage, prompt)

if isinstance(result, list):
    for r in result:
        print(r)
else:
    print(result)